In [1]:
import pandas as pd
import numpy as np

In [36]:
商品 = "ZN"

IB09_df = pd.read_parquet(f"datas/{商品}/IB_{商品}_5min-20240701-20240919.pqt")
IB12_df = pd.read_parquet(f"datas/{商品}/IB_{商品}_5min-20240923-20241030.pqt")
IB09_df["datetime"] = pd.to_datetime(IB09_df.index)
IB12_df["datetime"] = pd.to_datetime(IB12_df.index)

In [37]:
# 合併IB
# 移除IB09_df['Datetime']不在15:00:00-04:00:00的row
IB09_df = IB09_df[
    (IB09_df["datetime"].dt.time >= pd.to_datetime("15:00:00").time())
    | (IB09_df["datetime"].dt.time <= pd.to_datetime("04:00:00").time())
]
# 移除IB12_df['Datetime']不在15:00:00-04:00:00的row
IB12_df = IB12_df[
    (IB12_df["datetime"].dt.time >= pd.to_datetime("15:00:00").time())
    | (IB12_df["datetime"].dt.time <= pd.to_datetime("04:00:00").time())
]
# 合併IB09_df和IB12_df
IB_df = pd.concat([IB09_df, IB12_df])

# 移除IB_df['datetime']重複的row
IB_df = IB_df.drop_duplicates(subset=["datetime"])

關於直接整理ES 5min資料集

In [ ]:
RTY_df = IB_df.copy()
# 假設你已有 DataFrame 'df'，並包含 'close' 欄位以及以5分鐘為間隔的索引
# ES_df["ES 5分鐘震幅"] = (ES_df["high"] - ES_df["low"]) * 10000 / ES_df["low"]
# 新增5分鐘、10分鐘、30分鐘、60分鐘的漲跌幅
# ES_df["ES 5分鐘漲跌幅"] = (ES_df["close"] - ES_df["open"]) * 10000 / ES_df["close"]
# ES_df["ES 10分鐘漲跌幅"] = ES_df["close"].pct_change(periods=2) * 10000  # 10分鐘

RTY_df[f"{商品} 30分鐘漲跌幅"] = RTY_df["close"].pct_change(periods=6) * 10000  # 30分鐘

RTY_df[f"{商品} 60分鐘漲跌幅"] = (
    RTY_df["close"].pct_change(periods=12) * 10000
)  # 60分鐘

# 新增30分鐘和60分鐘的平均收盤價距離現在的漲跌幅
# ES_df["ES 30分鐘平均的close比"] = (ES_df["close"]-ES_df["close"].rolling(window=6).mean() )*10000 / ES_df["close"]

# ES_df["ES 60分鐘平均的close比"] = (
#     (ES_df["close"] - ES_df["close"].rolling(window=12).mean()) * 10000 / ES_df["close"]
# )
# 處理成交量
# ES_df["ES 60分鐘平均的volume"] = (
#     (ES_df["volume"] - ES_df["volume"].rolling(window=6).mean())
# )
# ES_df["ES 60分鐘平均的count"] = (
#     (ES_df["count"] - ES_df["count"].rolling(window=6).mean())
# )
# ES_df["ES 每筆成交量"] = ES_df["volume"] / ES_df["count"]

RTY_df=RTY_df.drop(["high", "low", "open","datetime"], axis=1)


RTY_df = RTY_df.rename(
    {

        "open": f"{商品} open",
        "close": f"{商品} close",
        "volume": f"{商品} volume",
        "count": f"{商品} count",
    },
    axis="columns",
)

RTY_df

,ZN close,ZN volume,ZN count,ZN 30分鐘漲跌幅,ZN 60分鐘漲跌幅
datetime,,,,,
2024-05-22 15:00:00,109.312500,328,47,NaN,NaN
2024-05-22 15:05:00,109.343750,254,35,NaN,NaN
2024-05-22 15:10:00,109.343750,72,18,NaN,NaN
2024-05-22 15:15:00,109.328125,199,37,NaN,NaN
2024-05-22 15:20:00,109.312500,138,32,NaN,NaN
...,...,...,...,...,...
2024-10-31 03:40:00,110.593750,10728,592,2.826456,-9.880028
2024-10-31 03:45:00,110.562500,10962,252,1.413428,-14.112334
2024-10-31 03:50:00,110.546875,7829,644,-1.413228,-16.932411


In [39]:
RTY_df.to_csv(f"datas/{商品}/{商品}_0701-1030_5min含筆數與張數.csv")

關於將YM 1min資料集合並成5min資料集

In [17]:
# 將數據重新取樣為每5分鐘
def resample_5min(df):
    # 計算每5分鐘的成交總張數、成交總筆數
    df_resampled = df.resample("5T").agg(
        {"volume": ["sum", "mean", "max", "min"], "count": ["sum", "std"]}
    )
    df_resampled.columns = [
        "YM五分成交總量",
        "YM五分內平均1min成交量",
        "YM五分內最大1min成交量",
        "YM五分內最小1min成交量",
        "YM五分內成交總筆數",
        "YM五分內成交筆數標準差",
    ]

    # 計算每筆成交的平均張數
    df_resampled["YM五分內平均每筆張數"] = (
        df_resampled["YM五分成交總量"] / df_resampled["YM五分內成交總筆數"]
    )

    # 計算大於120張的成交次數與大於80筆的成交次數
    df_resampled["YM五分內每min大於120張的次數"] = (
        df["volume"].resample("5T").apply(lambda x: (x > 120).sum())
    )
    df_resampled["YM五分內每min大於80筆的次數"] = (
        df["count"].resample("5T").apply(lambda x: (x > 80).sum())
    )

    # 處理可能的 NaN 值
    df_resampled.fillna(0, inplace=True)

    return df_resampled


df_5min = resample_5min(IB_df)
df_5min

,YM五分成交總量,YM五分內平均1min成交量,YM五分內最大1min成交量,YM五分內最小1min成交量,YM五分內成交總筆數,YM五分內成交筆數標準差,YM五分內平均每筆張數,YM五分內每min大於120張的次數,YM五分內每min大於80筆的次數
datetime,,,,,,,,,
2024-05-27 15:00:00,0,0.0,0.0,0.0,0,0.000000,0.000000,0,0
2024-05-27 15:05:00,2,0.4,2.0,0.0,2,0.894427,1.000000,0,0
2024-05-27 15:10:00,0,0.0,0.0,0.0,0,0.000000,0.000000,0,0
2024-05-27 15:15:00,0,0.0,0.0,0.0,0,0.000000,0.000000,0,0
2024-05-27 15:20:00,0,0.0,0.0,0.0,0,0.000000,0.000000,0,0
...,...,...,...,...,...,...,...,...,...
2024-10-31 03:40:00,766,153.2,202.0,106.0,516,19.992499,1.484496,4,4
2024-10-31 03:45:00,889,177.8,208.0,148.0,591,15.770225,1.504230,5,5
2024-10-31 03:50:00,1459,291.8,490.0,140.0,928,73.656636,1.572198,5,5


In [16]:
df_5min.describe()

,volume_sum,volume_avg,volume_max,volume_min,count_sum,count_std,avg_volume_per_trade,volume_over_120_count,count_over_80_count
count,45085.000000,45085.000000,45085.000000,45085.000000,45085.000000,45085.000000,45085.000000,45085.000000,45085.000000
mean,244.643562,50.446805,79.816524,31.189775,165.614439,11.731248,0.528181,0.613951,0.633714
std,643.284076,135.367750,223.746109,92.533359,431.764772,36.829002,0.705737,1.492133,1.533808
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,154.000000,31.000000,52.000000,15.000000,110.000000,8.497058,1.356522,0.000000,0.000000
max,11688.000000,2337.600000,5219.000000,2318.000000,7966.000000,789.339154,5.857143,5.000000,5.000000


In [18]:
df_5min.to_csv("datas/YM/YM_0701-1030_5min含筆數與張數.csv")